In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

In [ ]:
lncrna=pd.read_csv('/bin/app/model/X_train_lncrna.csv')
lncrna["lncRNA"] = True
not_lncrna=pd.read_csv('/bin/app/model/X_train_not_lncrna.csv')
not_lncrna["lncRNA"] = False
lncrna.drop(["mfe"], axis=1, inplace=True)
not_lncrna.drop(["none"], axis=1, inplace=True)
features = pd.DataFrame({})
features = pd.concat([lncrna, not_lncrna], ignore_index=True)
features.sort_values(ascending=True, by=["start", "end", "name"], inplace=True)
features_clean = features.dropna(axis=0, how="any")
dropped = 1 - len(features_clean) / len(features)
print('Dropped ' + str(round(dropped, 3) * 100) + '% of rows from the original dataset due to NA values')
X = features_clean.iloc[:,3:14].values
y = features_clean.iloc[:,14].values

In [ ]:
rf = RandomForestClassifier()
estimators = list(range(1,11))
param_grid = {
    'n_estimators' : estimators,
    'max_depth' : [3, 4, 5, 6, 7, 8, 9],
    'min_samples_split' : [2, 4, 6, 8, 10],
    'min_samples_leaf' : [1, 2, 5, 10]
}
gs = GridSearchCV(rf, param_grid, scoring='precision', verbose=2, n_jobs=4)
gs.fit(X, y)

In [17]:
gs.best_params_


{'max_depth': 8,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 10}

In [19]:
rf_best = RandomForestClassifier(n_estimators=10, max_depth=8, min_samples_leaf=5, min_samples_split=10)

from sklearn.model_selection import train_test_split
Xtr, Xts, Ytr, Yts = train_test_split(X, y, random_state=15)
Xtr.shape, Xts.shape, Ytr.shape, Yts.shape

((172749, 11), (57584, 11), (172749,), (57584,))

In [22]:
rf_best.fit(Xtr, Ytr)
Ypr = rf_best.predict(Xts)
accuracy_score(Yts, Ypr)

0.9670568213392609